In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../data/humob/humob_processed.csv")

In [3]:
trajectory_length = 48

# Agregar una columna para `p_i`
data['p_i'] = 20 * (data['x'] - 1) + data['y']

# Crear listas para almacenar resultados
trajectories = []
user_indices = []
days = []

# Agrupar por `uid` y `d`
grouped = data.groupby(['uid', 'd'])

for (uid, day), group in grouped:
    if len(group) < 10:
        continue # Ignorar grupos con menos de 10 registros
    
    # Crear un array para toda la trayectoria inicializado en 'PAD'
    trajectory = np.full(trajectory_length, '[PAD]', dtype=object)
    
    # Asignar los valores de `p_i` según el índice de `t`
    trajectory[group['t'].values] = group['p_i'].astype(str).values
    
    # Guardar resultados
    trajectories.append(trajectory.tolist())
    user_indices.append(uid)
    days.append(day)

In [4]:
df = pd.DataFrame({
    'trajectory': trajectories,
    'user_index': user_indices,
    'day': days
})

In [5]:
# Guardar como HDF5
h5_file = "humob.h5"
df.to_hdf(h5_file, key='data', mode='w')

C:\Users\alexs\AppData\Local\Temp\ipykernel_74596\1694227914.py:3: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['trajectory'], dtype='object')]

  df.to_hdf(h5_file, key='data', mode='w')
